In [1]:
from collections import defaultdict

# Function to calculate the epsilon closure of a set of states.
def epsilon_closure(nfa, states):
    stack = list(states)  # Stack for states that need to be processed.
    closure = set(states) # Set to store the closure states.

    # While there are states to process.
    while stack:
        state = stack.pop() # Pop a state from the stack.
        # If there is an epsilon transition from this state.
        if (state, '') in nfa:
            # For each state reachable from the current state through an epsilon transition.
            for next_state in nfa[(state, '')]:
                # If the state is not already in the closure set.
                if next_state not in closure:
                    closure.add(next_state) # Add it to the closure set.
                    stack.append(next_state) # And to the stack to process its transitions.

    return closure # Return the set of states in the epsilon closure.

# Function to convert NFA to DFA.
def nfa_to_dfa(nfa, start_state, accept_states):
    dfa = defaultdict(list) # The DFA transition function.
    # Compute the epsilon closure of the start state to get the DFA's start state.
    dfa_start_state = '+'.join(epsilon_closure(nfa, [start_state]))
    dfa_accept_states = [] # List to store the DFA's accept states.
    unmarked_states = [dfa_start_state] # States that need to be processed.
    dfa_states = set([dfa_start_state]) # Set of DFA states that have been processed.

    # While there are unmarked states.
    while unmarked_states:
        current_dfa_state = unmarked_states.pop() # Get one unmarked state.
        nfa_states = current_dfa_state.split('+') # Split to get the individual NFA states.

        # For each possible input symbol (excluding epsilon).
        for symbol in set(sym for state, sym in nfa if sym):
            next_states = set() # Set to store the next states.

            # For each NFA state in the current DFA state.
            for state in nfa_states:
                # If there is a transition from the NFA state on the symbol.
                if (state, symbol) in nfa:
                    # For each state reachable from the NFA state on the symbol.
                    for next_state in nfa[(state, symbol)]:
                        # Add the states in the epsilon closure of the next state.
                        next_states |= epsilon_closure(nfa, [next_state])

            # If there are next states.
            if next_states:
                # Name the next DFA state by joining the names of the next states.
                next_dfa_state = '+'.join(next_states)
                # Add the transition to the DFA.
                dfa[(current_dfa_state, symbol)] = [next_dfa_state]

                # If the next state is a new state.
                if next_dfa_state not in dfa_states:
                    dfa_states.add(next_dfa_state) # Add it to the set of DFA states.
                    unmarked_states.append(next_dfa_state) # And to the list of unmarked states.

    # For each state in the DFA.
    for dfa_state in dfa_states:
        # If any NFA state in the DFA state is an accept state.
        if any(nfa_state in accept_states for nfa_state in dfa_state.split('+')):
            # Mark the DFA state as an accept state (if not already marked).
            if not dfa_state.startswith('*'):
                dfa_state = '*' + dfa_state
            dfa_accept_states.append(dfa_state)

    # Return the DFA, its start state, and its accept states.
    return dfa, dfa_start_state, dfa_accept_states

# Example NFA (transitions are stored in a dictionary with (state, symbol) as the key).
nfa = {
    ('q1', ''): ['q2', 'q4'],  # epsilon transitions
    ('q2', 'a'): ['q3'],
    ('q3', 'a'): ['q2'],
    ('q4', 'a'): ['q5'],
    ('q5', 'a'): ['q6'],
    ('q6', 'a'): ['q4'],
}

start_state = 'q1' # The start state of the NFA.
accept_states = set(['q2', 'q6']) # The accept states of the NFA.

# Convert the NFA to a DFA.
dfa, dfa_start_state, dfa_accept_states = nfa_to_dfa(nfa, start_state, accept_states)

# Print the DFA's transition function, start state, and accept states.
print("DFA Transitions:")
for key, value in dfa.items():
    print(f"{key}: {value}")
print(f"DFA Start State: {dfa_start_state}")
print(f"DFA Accept States: {dfa_accept_states}")

DFA Transitions:
('q2+q1+q4', 'a'): ['q3+q5']
('q3+q5', 'a'): ['q2+q6']
('q2+q6', 'a'): ['q3+q4']
('q3+q4', 'a'): ['q2+q5']
('q2+q5', 'a'): ['q3+q6']
('q3+q6', 'a'): ['q2+q4']
('q2+q4', 'a'): ['q3+q5']
DFA Start State: q2+q1+q4
DFA Accept States: ['*q2+q1+q4', '*q2+q6', '*q2+q4', '*q3+q6', '*q2+q5']
